In [ ]:
import pandas as pd
from tqdm import tqdm
from yandex_reviews_parser import YandexParser

# Файл для промежуточного сохранения данных
CHECKPOINT_FILE = r"checkpoints/reviews_checkpoint.xlsx"

# Загрузка таблицы с данными организаций из Excel-файла.
df_org = pd.read_excel("organizations.xlsx")

all_reviews = []

# Функция для сохранения промежуточных данных
def save_checkpoint(data, filename):
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)
    print(f"Промежуточное сохранение: {len(data)} записей сохранено в {filename}")

# Перебираем организации с отображением прогресса
for idx, row in tqdm(df_org.iterrows(), total=len(df_org), desc="Обработка организаций"):
    id_ya = row["id_ya"]
    try:
        parser = YandexParser(id_ya)  # Инициализация парсера для организации
        result = parser.parse()       # Получаем данные
        if result.get("error"):
            print(f"Ошибка для id {id_ya}: {result.get('error')}")
            continue

        company_info = result.get("company_info", {})
        reviews = result.get("company_reviews", [])

        # Для каждого отзыва создаём отдельную запись, добавляя данные компании
        for review in reviews:
            data_row = {
                "company_name": company_info.get("name"),
                "company_rating": company_info.get("rating"),
                "company_count_rating": company_info.get("count_rating"),
                "company_stars": company_info.get("stars"),
                "review_name": review.get("name"),
                "review_icon_href": review.get("icon_href"),
                "review_date": review.get("date"),
                "review_text": review.get("text"),
                "review_stars": review.get("stars"),
                "review_answer": review.get("answer")
            }
            all_reviews.append(data_row)
    except Exception as e:
        print(f"Ошибка при обработке id {id_ya}: {e}")
    
    # Периодически сохраняем промежуточный результат (например, каждые 5 организаций)
    if idx % 5 == 0:
        save_checkpoint(all_reviews, CHECKPOINT_FILE)
        # Если требуется, можно добавить задержку или логику повторной попытки

# Финальное сохранение всех данных в итоговый файл
df_reviews = pd.DataFrame(all_reviews)
df_reviews.to_excel(r"results/reviews.xlsx", index=False)
print(f"Данные сохранены в файл reviews.xlsx. Всего записей: {len(all_reviews)}")

Обработка организаций:  33%|███▎      | 1/3 [02:09<04:18, 129.18s/it]

Промежуточное сохранение: 600 записей сохранено в reviews_checkpoint.xlsx


Обработка организаций: 100%|██████████| 3/3 [05:18<00:00, 106.31s/it]


Данные сохранены в файл reviews.xlsx. Всего записей: 1537
